In [1]:
#r "nuget: Microsoft.SemanticKernel, *-*"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, *-*"

Installed Packages Microsoft.SemanticKernel, 1.0.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.0.1-preview

In [2]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Planning;


In [3]:
#!import ../../Plugins/CustomPlugin/CompanySearchPlugin.cs

In [4]:
using System;
using System.Collections.Generic;
using System.Diagnostics;
using System.IO;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;

In [5]:
Kernel kernel = Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion("Your Azure OpenAI Service Deployment Name" , "Your Azure OpenAI Service Endpoint", "Your Azure OpenAI Service API Key")
            .Build();

In [6]:
var companySearchPluginObj = new CompanySearchPlugin();
var companySearchPlugin = kernel.ImportPluginFromObject(companySearchPluginObj, "CompanySearchPlugin");

In [7]:
var pluginDirectory = Path.Combine("../..", "plugins");

In [8]:
var writetPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "WriterPlugin"));
var emailPlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "EmailPlugin"));
var translatePlugin =  kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginDirectory, "TranslatePlugin"));

In [9]:
string goal =  """
Check the weather in Guangzhou and generate dressing tips based on the results,  and write emails
""";

In [10]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

In [11]:
#pragma warning disable SKEXP0060

var originalPlan = await planner.CreatePlanAsync(kernel, goal);

In [12]:
Console.WriteLine(originalPlan);

{{!-- Step 1: Search the weather in Guangzhou --}}
{{set "weather" (CompanySearchPlugin-WeatherSearch text="Guangzhou")}}

{{!-- Step 2: Generate dressing tips based on the weather --}}
{{set "tips" (WriterPlugin-Tips input=weather language="en")}}

{{!-- Step 3: Write an email with the dressing tips --}}
{{set "email" (EmailPlugin-WeatherMail input=tips)}}

{{!-- Step 4: Print the email to the screen --}}
{{json email}}


In [13]:
originalPlan.ToString()

{{!-- Step 1: Search the weather in Guangzhou --}}
{{set "weather" (CompanySearchPlugin-WeatherSearch text="Guangzhou")}}

{{!-- Step 2: Generate dressing tips based on the weather --}}
{{set "tips" (WriterPlugin-Tips input=weather language="en")}}

{{!-- Step 3: Write an email with the dressing tips --}}
{{set "email" (EmailPlugin-WeatherMail input=tips)}}

{{!-- Step 4: Print the email to the screen --}}
{{json email}}

In [14]:
#pragma warning disable SKEXP0060
var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

In [15]:
Console.WriteLine(originalPlanResult);

Subject: Dressing Tips for the Rainy Weather in Guangzhou

Dear [Colleague's Name],

I hope this email finds you well. I wanted to give you some dressing tips for the current weather conditions in Guangzhou, as it is currently 2 degrees Celsius and raining. It is important to dress appropriately to stay warm and comfortable during your commute.

1. Choose a heavy coat: I recommend wearing a thick coat, such as a down jacket or a woolen coat. This will help to keep your body warm and protected from the cold rain.

2. Layer your clothes: To provide extra insulation, wear a long-sleeved sweater or shirt under your top. This will help to trap heat and keep you warm.

3. Wear long pants or thick skirts: Protect your legs from the cold by wearing long pants or thick skirts. This will help to maintain your body temperature and prevent any discomfort caused by the cold and rain.

4. Don't forget your hat and gloves: Keep your head and hands warm by wearing a hat and gloves. These accessories a